In [ ]:
def filter_reference(i):
    min_refl   = 0.
    max_refl   = 30.
    
    #extract refl data
    refl_img_full = refl_data[i+200, :,:]

    #crop to valid data
    refl_img_crop = refl_img_full[:, x_start:] 

    #discreise reflectivity
    refl_img_int = refl_to_int(refl_img_crop)

    #valid reflectivity mask
    valid_refl_mask = np.logical_and(refl_img_int>=min_refl, refl_img_int<=max_refl)

    #g1 and g2 convolution filers filter
    conv_g1    = ndimage.uniform_filter(refl_img_int, size=g1_size, mode='constant', cval=pad_value)
    conv_g2    = ndimage.uniform_filter(refl_img_int, size=g2_size, mode='constant', cval=pad_value)

    #foreground mask
    inital_mask = np.logical_and(refl_img_int >= conv_g1, refl_img_int > conv_g2)
    ridges_mask = np.logical_and(inital_mask, valid_refl_mask)
    
    
    #smooth ridge edges to remove burrs using dilation using a 3x3 filter
    dilation_filter = np.array([[0,1,0],[1,1,1],[0,1,0]], np.uint8)
    ridges_mask_smooth = morphology.binary_dilation(ridges_mask, dilation_filter)

    #create narrow bands
    skeleton = morphology.thin(ridges_mask_smooth)
    
    #plotting
    plt.clf()
    fig = plt.figure(facecolor='white', figsize=(30, 15))

    ax0 = plt.subplot(1,2,1)

    im0 = ax0.imshow(refl_img_crop.astype('float'))
    [line_i, line_j] = np.where(skeleton)
    ax0.plot(line_j, line_i, 'w.', markersize=makersize)
    ax0.set_title('Reflectivity')
    fig.colorbar(im0, ax=ax0)
    
    if zoom:
        ax0.set_ylim(zoom_y)
        ax0.set_xlim(zoom_x)
        
    ax1 = plt.subplot(1,2,2)
    im1 = ax1.imshow(refl_img_crop.astype('float'))
    
    if zoom:
        ax1.set_ylim(zoom_y)
        ax1.set_xlim(zoom_x)